In [1]:
!pip install -q tensorflow-gpu==2.0.0-alpha0

    100% |████████████████████████████████| 332.1MB 56kB/s 
    100% |████████████████████████████████| 3.0MB 6.4MB/s 
    100% |████████████████████████████████| 61kB 30.0MB/s 
    100% |████████████████████████████████| 419kB 4.5MB/s 


In [2]:
!pip install sacrebleu # https://github.com/mjpost/sacreBLEU

  Stored in directory: /root/.cache/pip/wheels/40/f1/a9/4cce0ec602e8d195da27bb9b8f6708ec778fbafdbabb097fde
Successfully built sacrebleu


In [3]:
!wget http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip

--2019-04-25 22:01:25--  http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 209.85.234.128, 2607:f8b0:4001:c1b::80
Connecting to storage.googleapis.com (storage.googleapis.com)|209.85.234.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2638744 (2.5M) [application/zip]
Saving to: ‘spa-eng.zip’

spa-eng.zip         100%[===================>]   2.52M  --.-KB/s    in 0.02s   

2019-04-25 22:01:25 (166 MB/s) - ‘spa-eng.zip’ saved [2638744/2638744]



In [4]:
!pwd
zipfile1 = '/content/spa-eng.zip'

/content


In [0]:
import zipfile
zip_ref = zipfile.ZipFile(zipfile1, 'r')
zip_ref.extractall('/content')
zip_ref.close()

# Part 1

In [55]:
# !ls ./spa-eng/
# !cat ./spa-eng/spa.txt

sentences = []
counter = 0
with open("./spa-eng/spa.txt",'r') as f:
    for line in f:
        counter += 1
        if counter == 4001:
          break
        line = line.split('\t')
        sentences.append((line[0],line[1]))


print(len(sentences))
npsen = np.array(sentences)
print(npsen.shape)
# print(sentences)

4000
(4000, 2)


In [56]:
import numpy as np
import re
import sacrebleu
import tensorflow as tf
import time
import unicodedata

# sentences = [
#   ("Do you want a cup of coffee?", "¿Quieres una taza de café?"),
#   ("I've had coffee already.", "Ya tomé café."),
#   ("Can I get you a coffee?", "¿Quieres que te traiga un café?"),
#   ("Please give me some coffee.", "Dame algo de café por favor."),
#   ("Would you like me to make coffee?", "¿Quieres que prepare café?"),
#   ("Two coffees, please.", "Dos cafés, por favor."),
#   ("How about a cup of coffee?", "¿Qué tal una taza de café?"),
#   ("I drank two cups of coffee.", "Me tomé dos tazas de café."),
#   ("Would you like to have a cup of coffee?", "¿Te gustaría tomar una taza de café?"),
#   ("There'll be coffee and cake at five.", "A las cinco habrá café y un pastel."),
#   ("Another coffee, please.", "Otro café, por favor."),
#   ("I made coffee.", "Hice café."),
#   ("I would like to have a cup of coffee.", "Quiero beber una taza de café."),
#   ("Do you want me to make coffee?", "¿Quieres que haga café?"),
#   ("It is hard to wake up without a strong cup of coffee.", "Es difícil despertarse sin una taza de café fuerte."),
#   ("All I drank was coffee.", "Todo lo que bebí fue café."),
#   ("I've drunk way too much coffee today.", "He bebido demasiado café hoy."),
#   ("Which do you prefer, tea or coffee?", "¿Qué prefieres, té o café?"),
#   ("There are many kinds of coffee.", "Hay muchas variedades de café."),
#   ("I will make some coffee.",	"Prepararé algo de café.")
# ]
npsen = np.array(sentences)
print(npsen.shape)
def preprocess(s):
  # for details, see https://www.tensorflow.org/alpha/tutorials/sequences/nmt_with_attention
  s = ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')
  s = re.sub(r"([?.!,¿])", r" \1 ", s)
  s = re.sub(r'[" "]+', " ", s)
  s = re.sub(r"[^a-zA-Z?.!,¿]+", " ", s)
  s = s.strip()
  s = '<start> ' + s + ' <end>'
  return s


print("Original:", sentences[0])
sentences = [(preprocess(source), preprocess(target)) for (source, target) in sentences]
print("Preprocessed:", sentences[0])

(4000, 2)
Original: ('Go.', 'Ve.\n')
Preprocessed: ('<start> Go . <end>', '<start> Ve . <end>')


In [58]:
source_sentences, target_sentences = list(zip(*sentences))
source_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
source_tokenizer.fit_on_texts(source_sentences)
source_data = source_tokenizer.texts_to_sequences(source_sentences)
print("Sequence:", source_data[0])
source_data = tf.keras.preprocessing.sequence.pad_sequences(source_data, padding='post')
print("Padded:", source_data[0])
target_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
target_tokenizer.fit_on_texts(target_sentences)
target_data = target_tokenizer.texts_to_sequences(target_sentences)
target_data = tf.keras.preprocessing.sequence.pad_sequences(target_data, padding='post')

# Create labels for the decoder by shifting the target sequence
# one to the right.
target_labels = np.zeros(target_data.shape)
target_labels[:,0:target_data.shape[1] -1] = target_data[:,1:]

print("Target sequence", target_data[0])
print("Target label", target_labels[0])

source_vocab_size = len(source_tokenizer.word_index) + 1
target_vocab_size = len(target_tokenizer.word_index) + 1

def decode(encoded, tokenizer):
  for number in encoded:
    if number !=0:
      print ("%d -> %s" % (number, tokenizer.index_word[number]))
      
decode(source_data[0], source_tokenizer)

batch_size = 5
dataset = tf.data.Dataset.from_tensor_slices((source_data, target_data, target_labels)).batch(batch_size)

example_batch = next(iter(dataset))
source, target, taget_labels = example_batch
print("Shapes:", source.shape, target.shape, taget_labels.shape)

embedding_size = 32
rnn_size = 64

class Encoder(tf.keras.Model):
  def __init__(self):
    super(Encoder, self).__init__()
    
    self.embedding = tf.keras.layers.Embedding(source_vocab_size,
                                               embedding_size)
    self.gru = tf.keras.layers.GRU(rnn_size, 
                                   return_sequences=True, 
                                   return_state=True)
    
  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state=hidden)        
    return output, state
  
  def init_state(self, batch_size):
    return tf.zeros((batch_size, rnn_size))
  
  
# Create a batch of one sentence
ex_sentence = tf.expand_dims(source_data[0], axis=0)
ex_translation = tf.expand_dims(target_data[0], axis=0)
ex_labels = tf.expand_dims(target_labels[0], axis=0)
print(ex_sentence.shape)

encoder = Encoder()
hidden_state = encoder.init_state(batch_size=1)
print(hidden_state.shape)

output, hidden_state = encoder(ex_sentence, hidden_state)
print(output.shape)


class Decoder(tf.keras.Model):
  def __init__(self):
    super(Decoder, self).__init__()
    self.embedding = tf.keras.layers.Embedding(target_vocab_size, 
                                               embedding_size)
    self.gru = tf.keras.layers.GRU(rnn_size, 
                                   return_sequences=True, 
                                   return_state=True)

    self.dense = tf.keras.layers.Dense(target_vocab_size)


  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state=hidden)
    logits = self.dense(output)
    return logits, state
  

decoder = Decoder()
decoder_output, decoder_state = decoder(ex_labels, hidden_state)
print(decoder_output.shape)

crossentropy = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

def calc_loss(targets, logits):
  mask = tf.math.logical_not(tf.math.equal(targets, 0))
  mask = tf.cast(mask, dtype=tf.int64)
  return crossentropy(targets, logits, sample_weight=mask)

print("Loss", calc_loss(ex_labels, decoder_output))


def translate(idx=None):
  
    if idx == None: 
      idx = np.random.choice(len(sentences))
    
    input_sent = source_data[idx]
    input_sent = tf.expand_dims(input_sent, axis=0)
    
    hidden_state = encoder.init_state(batch_size=1)
    output, hidden_state = encoder(input_sent, hidden_state)
    
    decoder_input = tf.expand_dims([target_tokenizer.word_index['<start>']], 0)
    out_words = []
    
    decoder_state = hidden_state

    while True:
      
        decoder_output, decoder_state = decoder(decoder_input, decoder_state)
        decoder_input = tf.argmax(decoder_output, -1)
        word_idx = decoder_input.numpy()[0][0]
        # if we've predicted 0 (which is reserved, usually this will only happen
        # before the decoder is trained, just stop translating and return
        # what we have)
        if word_idx == 0: 
          out_words.append('<end>')
        else:
          out_words.append(target_tokenizer.index_word[word_idx])

        if out_words[-1] == '<end>' or len(out_words) >= 20:
          break
          
    translation = ' '.join(out_words)    
    return sentences[idx][0], sentences[idx][1], translation
  
input_sent, target_sent, translation = translate()
print("Input: %s\nTarget: %s\nTranslation: %s\n" % (input_sent, target_sent, translation))

optimizer = tf.keras.optimizers.Adam()
@tf.function # remove this annotation when debugging
def train_step(source_seq, target_seq, target_labels, initial_state):
  
  with tf.GradientTape() as tape:
    encoder_output, encoder_state = encoder(source_seq, initial_state)
    logits, decoder_state = decoder(target_seq, encoder_state)
    loss = calc_loss(target_labels, logits)

  variables = encoder.trainable_variables + decoder.trainable_variables
  gradients = tape.gradient(loss, variables)
  optimizer.apply_gradients(zip(gradients, variables))

  return loss

EPOCHS = 300

for epoch in range(EPOCHS):
    start = time.time()
  
    en_initial_states = encoder.init_state(batch_size)
    
    for batch, (source_seq, target_seq, target_labels) in enumerate(dataset):
      loss = train_step(source_seq, target_seq, target_labels, en_initial_states)
      elapsed = time.time() - start
    
    if epoch % 10 == 0:
      print("Epoch #%d, Loss %.4f, Time %.2f sec" % (epoch, loss, elapsed))
      input_sent, target_sent, translation = translate()
      print("Input: %s\nTarget: %s\nTranslation: %s\n" % (input_sent, target_sent, translation))
      
      
references, hypotheses = [], []

for i in range(len(sentences)):
  input_sent, target_sent, translation = translate()
  references.append(target_sent)
  hypotheses.append("<start> " + translation)
  
results = sacrebleu.raw_corpus_bleu(hypotheses, [references])
print(results)

Sequence: [1, 18, 3, 2]
Padded: [ 1 18  3  2  0  0  0  0]
Target sequence [ 1 62  3  2  0  0  0  0  0  0  0  0]
Target label [62.  3.  2.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
1 -> <start>
18 -> go
3 -> .
2 -> <end>
Shapes: (5, 8) (5, 12) (5, 12)
(1, 8)
(1, 64)
(1, 8, 64)
(1, 12, 2376)
Loss tf.Tensor(1.9435749, shape=(), dtype=float32)
Input: <start> Have a seat . <end>
Target: <start> Tome asiento . <end>
Translation: brazo teje encendida participar consigueme lunes hazmelo apagalo magico tenerlo bebio caliente miralo conozco cambiamos centrarte escribe ataque ayudanos abri

Epoch #0, Loss 1.8491, Time 7.20 sec
Input: <start> Tom lost . <end>
Target: <start> Tom perdio . <end>
Translation: ¿ ¿ . <end>

Epoch #10, Loss 0.8980, Time 5.67 sec
Input: <start> Look at this . <end>
Target: <start> Fijate en esto . <end>
Translation: deja la segundo . <end>

Epoch #20, Loss 0.6048, Time 4.95 sec
Input: <start> She s pretty . <end>
Target: <start> Es guapa . <end>
Translation: es tranquila . <e

# Part 2

In [61]:


# !ls ./spa-eng/
# !cat ./spa-eng/spa.txt

sentences = []

sentences = []
counter = 0
with open("./spa-eng/spa.txt",'r') as f:
    for line in f:
        counter += 1
        if counter == 4001:
          break
        line = line.split('\t')
        sentences.append((line[1], line[0]))

print(sentences[2])


print("Original:", sentences[0])
sentences = [(preprocess(source), preprocess(target)) for (source, target) in sentences]
print("Preprocessed:", sentences[0])

source_sentences, target_sentences = list(zip(*sentences))
source_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
source_tokenizer.fit_on_texts(source_sentences)
source_data = source_tokenizer.texts_to_sequences(source_sentences)
print("Sequence:", source_data[0])
source_data = tf.keras.preprocessing.sequence.pad_sequences(source_data, padding='post')
print("Padded:", source_data[0])
target_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='')
target_tokenizer.fit_on_texts(target_sentences)
target_data = target_tokenizer.texts_to_sequences(target_sentences)
target_data = tf.keras.preprocessing.sequence.pad_sequences(target_data, padding='post')

# Create labels for the decoder by shifting the target sequence
# one to the right.
target_labels = np.zeros(target_data.shape)
target_labels[:,0:target_data.shape[1] -1] = target_data[:,1:]

print("Target sequence", target_data[0])
print("Target label", target_labels[0])

source_vocab_size = len(source_tokenizer.word_index) + 1
target_vocab_size = len(target_tokenizer.word_index) + 1

def decode(encoded, tokenizer):
  for number in encoded:
    if number !=0:
      print ("%d -> %s" % (number, tokenizer.index_word[number]))
      
decode(source_data[0], source_tokenizer)

batch_size = 5
dataset = tf.data.Dataset.from_tensor_slices((source_data, target_data, target_labels)).batch(batch_size)

example_batch = next(iter(dataset))
source, target, taget_labels = example_batch
print("Shapes:", source.shape, target.shape, taget_labels.shape)

embedding_size = 32
rnn_size = 64

class Encoder(tf.keras.Model):
  def __init__(self):
    super(Encoder, self).__init__()
    
    self.embedding = tf.keras.layers.Embedding(source_vocab_size,
                                               embedding_size)
    self.gru = tf.keras.layers.GRU(rnn_size, 
                                   return_sequences=True, 
                                   return_state=True)
    
  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state=hidden)        
    return output, state
  
  def init_state(self, batch_size):
    return tf.zeros((batch_size, rnn_size))
  
  
# Create a batch of one sentence
ex_sentence = tf.expand_dims(source_data[0], axis=0)
ex_translation = tf.expand_dims(target_data[0], axis=0)
ex_labels = tf.expand_dims(target_labels[0], axis=0)
print(ex_sentence.shape)

encoder = Encoder()
hidden_state = encoder.init_state(batch_size=1)
print(hidden_state.shape)

output, hidden_state = encoder(ex_sentence, hidden_state)
print(output.shape)


class Decoder(tf.keras.Model):
  def __init__(self):
    super(Decoder, self).__init__()
    self.embedding = tf.keras.layers.Embedding(target_vocab_size, 
                                               embedding_size)
    self.gru = tf.keras.layers.GRU(rnn_size, 
                                   return_sequences=True, 
                                   return_state=True)

    self.dense = tf.keras.layers.Dense(target_vocab_size)


  def call(self, x, hidden):
    x = self.embedding(x)
    output, state = self.gru(x, initial_state=hidden)
    logits = self.dense(output)
    return logits, state
  

decoder = Decoder()
decoder_output, decoder_state = decoder(ex_labels, hidden_state)
print(decoder_output.shape)

crossentropy = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

def calc_loss(targets, logits):
  mask = tf.math.logical_not(tf.math.equal(targets, 0))
  mask = tf.cast(mask, dtype=tf.int64)
  return crossentropy(targets, logits, sample_weight=mask)

print("Loss", calc_loss(ex_labels, decoder_output))


def translate2(idx=None):
  
    if idx == None: 
      idx = np.random.choice(len(sentences))
    
    input_sent = source_data[idx]
    input_sent = tf.expand_dims(input_sent, axis=0)
    
    hidden_state = encoder.init_state(batch_size=1)
    output, hidden_state = encoder(input_sent, hidden_state)
    
    decoder_input = tf.expand_dims([target_tokenizer.word_index['<start>']], 0)
    out_words = []
    
    decoder_state = hidden_state

    while True:
      
        decoder_output, decoder_state = decoder(decoder_input, decoder_state)
        decoder_input = tf.argmax(decoder_output, -1)
        word_idx = decoder_input.numpy()[0][0]
        # if we've predicted 0 (which is reserved, usually this will only happen
        # before the decoder is trained, just stop translating and return
        # what we have)
        if word_idx == 0: 
          out_words.append('<end>')
        else:
          out_words.append(target_tokenizer.index_word[word_idx])

        if out_words[-1] == '<end>' or len(out_words) >= 20:
          break
          
    translation = ' '.join(out_words)    
    return sentences[idx][0], sentences[idx][1], translation
  
input_sent, target_sent, translation = translate2()
print("Input: %s\nTarget: %s\nTranslation: %s\n" % (input_sent, target_sent, translation))


optimizer = tf.keras.optimizers.Adam()
@tf.function # remove this annotation when debugging
def train_step(source_seq, target_seq, target_labels, initial_state):
  
  with tf.GradientTape() as tape:
    encoder_output, encoder_state = encoder(source_seq, initial_state)
    logits, decoder_state = decoder(target_seq, encoder_state)
    loss = calc_loss(target_labels, logits)

  variables = encoder.trainable_variables + decoder.trainable_variables
  gradients = tape.gradient(loss, variables)
  optimizer.apply_gradients(zip(gradients, variables))

  return loss

EPOCHS = 100

for epoch in range(EPOCHS):
    start = time.time()
  
    en_initial_states = encoder.init_state(batch_size)
    
    for batch, (source_seq, target_seq, target_labels) in enumerate(dataset):
      loss = train_step(source_seq, target_seq, target_labels, en_initial_states)
      elapsed = time.time() - start
    
    if epoch % 10 == 0:
      print("Epoch #%d, Loss %.4f, Time %.2f sec" % (epoch, loss, elapsed))
      input_sent, target_sent, translation = translate2()
      print("Input: %s\nTarget: %s\nTranslation: %s\n" % (input_sent, target_sent, translation))
      
      
references, hypotheses = [], []

for i in range(len(sentences)):
  input_sent, target_sent, translation = translate2()
  references.append(target_sent)
  hypotheses.append("<start> " + translation)
  
results = sacrebleu.raw_corpus_bleu(hypotheses, [references])
print(results)



('Vaya.\n', 'Go.')
Original: ('Ve.\n', 'Go.')
Preprocessed: ('<start> Ve . <end>', '<start> Go . <end>')
Sequence: [1, 62, 3, 2]
Padded: [ 1 62  3  2  0  0  0  0  0  0  0  0]
Target sequence [ 1 18  3  2  0  0  0  0]
Target label [18.  3.  2.  0.  0.  0.  0.  0.]
1 -> <start>
62 -> ve
3 -> .
2 -> <end>
Shapes: (5, 12) (5, 8) (5, 8)
(1, 12)
(1, 64)
(1, 12, 64)
(1, 8, 1108)
Loss tf.Tensor(2.630156, shape=(), dtype=float32)
Input: <start> Llevate mi coche . <end>
Target: <start> Take my car . <end>
Translation: feast of bear naked gas feast of , goodbye closed stood agreed way relaxed answer answer went sick dancing twice

Epoch #0, Loss 2.4417, Time 7.34 sec
Input: <start> Agarra fuertemente . <end>
Target: <start> Hang on . <end>
Translation: you re a a a . <end>

Epoch #10, Loss 1.2688, Time 5.02 sec
Input: <start> No pares . <end>
Target: <start> Don t stop . <end>
Translation: don t be evil . <end>

Epoch #20, Loss 0.6649, Time 4.99 sec
Input: <start> Deja que Tom se quede . <end>
Ta

# Part 3

In [64]:
references, hypotheses = [], []

for i in range(len(sentences)):
  input_sent, target_sent, translation = translate()
  translation, input_sent, translation_back = translate2()
  references.append(input_sent)
  hypotheses.append("<start> " + translation_back)
  
results = sacrebleu.raw_corpus_bleu(hypotheses, [references])
print(results)

BLEU(score=31.825281588561914, counts=[16893, 7877, 3920, 1610], totals=[23509, 19509, 15509, 11509], precisions=[71.8575864562508, 40.376236608744684, 25.275646398865174, 13.989052046224694], bp=1.0, sys_len=23509, ref_len=23134)
